In [1]:
import pandas as pd
import numpy as np
# feature
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline

# models
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import torch

import warnings
warnings.filterwarnings("ignore")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/miniconda3/envs/nlp_cours/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/miniconda3/envs/nlp_cours/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/miniconda3/envs/nlp_cours/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/miniconda3/envs/nlp_cours/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_in

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [2]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

## Tâche1 : Prédire la venue des articles parmi [ACL | EMNLP | LREC]

In [3]:
train_venue = pd.read_csv('lishuxu-predict_venue-title+abstract/train-lishuxu-predict_venue-title+abstract.csv', sep=';')
train_venue.head()

,id,title,abstract,label
0,L12-1253,A Repository for the Sustainable Management of...,This paper presents the system architecture as...,LREC
1,2020.emnlp-main.526,Substance over Style: Document-Level Targeted ...,Existing language models excel at writing from...,EMNLP
2,2021.emnlp-main.317,To be Closer: Learning to Link up Aspects with...,Dependency parse trees are helpful for discove...,EMNLP
3,L12-1613,Constraint Based Description of Polish Multiwo...,We present an approach to the description of P...,LREC
4,2020.emnlp-main.135,Tired of Topic Models? Clusters of Pretrained ...,Topic models are a useful analysis tool to unc...,EMNLP


In [4]:
test_venue = pd.read_csv('lishuxu-predict_venue-title+abstract/test-lishuxu-predict_venue-title+abstract.csv', sep=';')
test_venue.head()

,id,title,abstract,label
0,D18-1415,A Teacher-Student Framework for Maintainable D...,Reinforcement learning (RL) is an attractive s...,EMNLP
1,2021.emnlp-main.746,Bridge to Target Domain by Prototypical Contra...,Zero-shot cross-domain slot filling alleviates...,EMNLP
2,2020.acl-main.181,What determines the order of adjectives in Eng...,We take up the scientific question of what det...,ACL
3,2021.emnlp-main.780,Exploiting Twitter as Source of Large Corpora ...,Semantic sentence embeddings are usually super...,EMNLP
4,2022.lrec-1.141,NyLLex: A Novel Resource of Swedish Words Anno...,"What makes a text easy to read or not, depends...",LREC


In [5]:
# préparer les données

train_venue['text'] = train_venue['title'] + ' ' + train_venue['abstract']
test_venue['text'] = test_venue['title'] + ' ' + test_venue['abstract']

X_train = train_venue['text']
y_train = train_venue['label']
X_test = test_venue['text']
y_test = test_venue['label']

In [11]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

tfidf_mnb: 0.5966666666666667
              precision    recall  f1-score   support

         ACL       0.49      0.44      0.46       500
       EMNLP       0.49      0.56      0.53       500
        LREC       0.82      0.79      0.81       500

    accuracy                           0.60      1500
   macro avg       0.60      0.60      0.60      1500
weighted avg       0.60      0.60      0.60      1500



In [14]:
# baseline1
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# countvec+LogREG
log_reg_count = LogisticRegression(max_iter=1000)
log_reg_count.fit(X_train_count, y_train)

y_pred_log_reg_count = log_reg_count.predict(X_test_count)
print("LogREG-big Accuracy:", accuracy_score(y_test, y_pred_log_reg_count))
print(classification_report(y_test, y_pred_log_reg_count))

LogREG-big Accuracy: 0.578
              precision    recall  f1-score   support

         ACL       0.48      0.47      0.48       500
       EMNLP       0.47      0.49      0.48       500
        LREC       0.79      0.77      0.78       500

    accuracy                           0.58      1500
   macro avg       0.58      0.58      0.58      1500
weighted avg       0.58      0.58      0.58      1500



In [15]:
# with MNB
tfidf_mnb = MultinomialNB()
tfidf_mnb.fit(X_train_tfidf, y_train)

y_pred_td_mnb = tfidf_mnb.predict(X_test_tfidf)
print("tfidf_mnb accuracy:", accuracy_score(y_test, y_pred_td_mnb))
print(classification_report(y_test, y_pred_td_mnb))

tfidf_mnb accuracy: 0.5966666666666667
              precision    recall  f1-score   support

         ACL       0.49      0.44      0.46       500
       EMNLP       0.49      0.56      0.53       500
        LREC       0.82      0.79      0.81       500

    accuracy                           0.60      1500
   macro avg       0.60      0.60      0.60      1500
weighted avg       0.60      0.60      0.60      1500



In [12]:
# 2. Régression logistique
log_reg_model = LogisticRegression(max_iter=1000)
log_reg_model.fit(X_train_tfidf, y_train)

y_pred_td_lr = log_reg_model.predict(X_test_tfidf)
print("tfidf_lr accuracy:", accuracy_score(y_test, y_pred_td_lr))
print(classification_report(y_test,  y_pred_td_lr))


tfidf_lr 0.6113333333333333
              precision    recall  f1-score   support

         ACL       0.51      0.46      0.48       500
       EMNLP       0.51      0.53      0.52       500
        LREC       0.80      0.84      0.82       500

    accuracy                           0.61      1500
   macro avg       0.61      0.61      0.61      1500
weighted avg       0.61      0.61      0.61      1500



In [29]:
td_rf = RandomForestClassifier()
td_rf.fit(X_train_tfidf, y_train)

y_pred_td_rf = td_rf.predict(X_test_tfidf)
print("tfidf_rf accuracy:", accuracy_score(y_test, y_pred_td_rf))
print(classification_report(y_test, y_pred_td_rf))

tfidf_rf accuracy: 0.614
              precision    recall  f1-score   support

         ACL       0.51      0.56      0.53       500
       EMNLP       0.53      0.45      0.49       500
        LREC       0.79      0.83      0.81       500

    accuracy                           0.61      1500
   macro avg       0.61      0.61      0.61      1500
weighted avg       0.61      0.61      0.61      1500



LogREG-big Accuracy: 0.606
              precision    recall  f1-score   support

         ACL       0.51      0.50      0.50       500
       EMNLP       0.51      0.48      0.49       500
        LREC       0.78      0.84      0.81       500

    accuracy                           0.61      1500
   macro avg       0.60      0.61      0.60      1500
weighted avg       0.60      0.61      0.60      1500



In [30]:
# td_SVM
td_svm_model = SVC()
td_svm_model.fit(X_train_tfidf, y_train)

y_pred_td_svm = td_svm_model.predict(X_test_tfidf)
print("tfidf_svm accuracy:", accuracy_score(y_test, y_pred_td_svm))
print(classification_report(y_test, y_pred_td_svm))

tfidf_svm accuracy: 0.61
              precision    recall  f1-score   support

         ACL       0.49      0.48      0.48       500
       EMNLP       0.52      0.52      0.52       500
        LREC       0.81      0.83      0.82       500

    accuracy                           0.61      1500
   macro avg       0.61      0.61      0.61      1500
weighted avg       0.61      0.61      0.61      1500



In [16]:
# countvec+MNB
mnb_model_count = MultinomialNB()
mnb_model_count.fit(X_train_count, y_train)

y_pred_mnb_count = mnb_model_count.predict(X_test_count)
print("count_mnb Accuracy:", accuracy_score(y_test, y_pred_mnb_count))
print(classification_report(y_test, y_pred_mnb_count))

count_mnb Accuracy: 0.602
              precision    recall  f1-score   support

         ACL       0.49      0.46      0.47       500
       EMNLP       0.51      0.53      0.52       500
        LREC       0.80      0.82      0.81       500

    accuracy                           0.60      1500
   macro avg       0.60      0.60      0.60      1500
weighted avg       0.60      0.60      0.60      1500



In [17]:
# countvec+SVM
svm_count = SVC()
svm_count.fit(X_train_count, y_train)

y_pred_svm_count = svm_count.predict(X_test_count)
print("count_mnb Accuracy:", accuracy_score(y_test, y_pred_svm_count))
print(classification_report(y_test, y_pred_svm_count))

count_mnb Accuracy: 0.62
              precision    recall  f1-score   support

         ACL       0.51      0.56      0.53       500
       EMNLP       0.54      0.48      0.51       500
        LREC       0.81      0.82      0.82       500

    accuracy                           0.62      1500
   macro avg       0.62      0.62      0.62      1500
weighted avg       0.62      0.62      0.62      1500



In [20]:
# count+RandomForest
count_rf= RandomForestClassifier()
count_rf.fit(X_train_count, y_train)

y_pred_rf_count = count_rf.predict(X_test_count)
print("count_rf Accuracy:", accuracy_score(y_test, y_pred_rf_count))
print(classification_report(y_test, y_pred_rf_count))

count_rf Accuracy: 0.6093333333333333
              precision    recall  f1-score   support

         ACL       0.52      0.53      0.53       500
       EMNLP       0.51      0.46      0.48       500
        LREC       0.76      0.84      0.80       500

    accuracy                           0.61      1500
   macro avg       0.60      0.61      0.60      1500
weighted avg       0.60      0.61      0.60      1500



#### Modifier des méta-paramètres

In [101]:
# 
count_vectorizer_modif = CountVectorizer(ngram_range=(1,2), max_features=15000, min_df=5, max_df=0.7)
X_train_count_modif = count_vectorizer_modif.fit_transform(X_train)
X_test_count_modif = count_vectorizer_modif.transform(X_test)

In [102]:
lg_to_modif = LogisticRegression(C=0.01, penalty='l2', solver='saga',max_iter=1000)
lg_to_modif.fit(X_train_count_modif, y_train)
y_pred_lg_modif = lg_to_modif.predict(X_test_count_modif)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lg_modif))
print(classification_report(y_test, y_pred_lg_modif))

Logistic Regression Accuracy: 0.6153333333333333
              precision    recall  f1-score   support

         ACL       0.51      0.47      0.49       500
       EMNLP       0.51      0.53      0.52       500
        LREC       0.82      0.85      0.83       500

    accuracy                           0.62      1500
   macro avg       0.61      0.62      0.61      1500
weighted avg       0.61      0.62      0.61      1500



In [18]:
# Use Word2Vec to represent text
sentences = [text.split() for text in X_train]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# extract features from Word2Vec
def vectorize_text(text):
    words = text.split()
    vector = np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv], axis=0)
    return vector if vector is not None else np.zeros(100)

X_train_word2vec = np.array([vectorize_text(text) for text in X_train])
X_test_word2vec = np.array([vectorize_text(text) for text in X_test])

# with Logistic Regression
log_reg_model_word2vec = LogisticRegression(max_iter=1000)
log_reg_model_word2vec.fit(X_train_word2vec, y_train)

# predict
y_pred_log_reg_word2vec = log_reg_model_word2vec.predict(X_test_word2vec)
print("Word2Vec + Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log_reg_word2vec))
print(classification_report(y_test, y_pred_log_reg_word2vec))

Word2Vec + Logistic Regression Accuracy: 0.586
              precision    recall  f1-score   support

         ACL       0.49      0.47      0.48       500
       EMNLP       0.50      0.46      0.48       500
        LREC       0.74      0.83      0.78       500

    accuracy                           0.59      1500
   macro avg       0.58      0.59      0.58      1500
weighted avg       0.58      0.59      0.58      1500



In [19]:
# word2vec_svm
svm_word2vec = SVC()
svm_word2vec.fit(X_train_word2vec, y_train)

y_pred_svm_word2vec = svm_word2vec.predict(X_test_word2vec)
print("Word2Vec + SVM Accuracy:", accuracy_score(y_test, y_pred_svm_word2vec))
print(classification_report(y_test, y_pred_svm_word2vec))

Word2Vec + SVM Accuracy: 0.5826666666666667
              precision    recall  f1-score   support

         ACL       0.50      0.43      0.46       500
       EMNLP       0.49      0.49      0.49       500
        LREC       0.73      0.83      0.78       500

    accuracy                           0.58      1500
   macro avg       0.57      0.58      0.58      1500
weighted avg       0.57      0.58      0.58      1500



In [22]:
# w2v+rf
rf_word2vec = RandomForestClassifier()
rf_word2vec.fit(X_train_word2vec, y_train)

y_pred_rf_word2vec = rf_word2vec.predict(X_test_word2vec)
print("Word2Vec + RandomForest Accuracy:", accuracy_score(y_test, y_pred_rf_word2vec))
print(classification_report(y_test, y_pred_rf_word2vec))

Word2Vec + RandomForest Accuracy: 0.5626666666666666
              precision    recall  f1-score   support

         ACL       0.47      0.50      0.48       500
       EMNLP       0.46      0.38      0.42       500
        LREC       0.73      0.81      0.77       500

    accuracy                           0.56      1500
   macro avg       0.55      0.56      0.56      1500
weighted avg       0.55      0.56      0.56      1500



In [24]:
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)

In [25]:
def get_bert_embeddings_batch(texts):
    # ensure that the input is a list
    texts = texts.tolist()
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True).to(device)
    outputs = model(**inputs)
    
    # move tensor to cpu
    return outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()

# embedding
X_train_bert = np.vstack([get_bert_embeddings_batch(X_train[i:i+32]) for i in range(0, len(X_train), 32)])
X_test_bert = np.vstack([get_bert_embeddings_batch(X_test[i:i+32]) for i in range(0, len(X_test), 32)])

In [26]:
# bert+RandomForest
bert_rf = RandomForestClassifier()
bert_rf.fit(X_train_bert, y_train)

y_pred_bert_rf = bert_rf.predict(X_test_bert)
print("BERT + RandomForest Accuracy:", accuracy_score(y_test, y_pred_bert_rf))
print(classification_report(y_test, y_pred_bert_rf))

BERT + RandomForest Accuracy: 0.5926666666666667
              precision    recall  f1-score   support

         ACL       0.50      0.52      0.51       500
       EMNLP       0.50      0.46      0.48       500
        LREC       0.76      0.80      0.78       500

    accuracy                           0.59      1500
   macro avg       0.59      0.59      0.59      1500
weighted avg       0.59      0.59      0.59      1500



In [27]:
# svm+bert
svc_bert = SVC()
svc_bert.fit(X_train_bert, y_train)

y_pred_svc_bert = svc_bert.predict(X_test_bert)
print("BERT + SVM Accuracy:", accuracy_score(y_test, y_pred_svc_bert))
print(classification_report(y_test, y_pred_svc_bert))

BERT + SVM Accuracy: 0.6226666666666667
              precision    recall  f1-score   support

         ACL       0.52      0.55      0.53       500
       EMNLP       0.53      0.51      0.52       500
        LREC       0.83      0.81      0.82       500

    accuracy                           0.62      1500
   macro avg       0.63      0.62      0.62      1500
weighted avg       0.63      0.62      0.62      1500



In [28]:
# LogREG+bert
log_reg_bert = LogisticRegression(max_iter=1000)
log_reg_bert.fit(X_train_bert, y_train)

y_pred_log_reg_bert = log_reg_bert.predict(X_test_bert)
print("BERT + LogREG Accuracy:", accuracy_score(y_test, y_pred_log_reg_bert))
print(classification_report(y_test, y_pred_log_reg_bert))

BERT + LogREG Accuracy: 0.616
              precision    recall  f1-score   support

         ACL       0.51      0.51      0.51       500
       EMNLP       0.52      0.53      0.52       500
        LREC       0.82      0.81      0.82       500

    accuracy                           0.62      1500
   macro avg       0.62      0.62      0.62      1500
weighted avg       0.62      0.62      0.62      1500



In [36]:
results = {
    'Model': ['tfidf_mnb', 'tfidf_lr', 'tfidf_svm', 'tfidf_rf', 'count_lr', 'count_mnb', 'count_svm', 'count_rf', 'w2v_lr', 'w2v_svm','w2v_rf', 'bert_rf', 'bert_svm', 'bert_lr'],

    'Accuracy': [
            accuracy_score(y_test, y_pred_td_mnb),
            accuracy_score(y_test, y_pred_td_lr),
            accuracy_score(y_test, y_pred_td_svm),
            accuracy_score(y_test, y_pred_td_rf),
            accuracy_score(y_test, y_pred_log_reg_count),
            accuracy_score(y_test, y_pred_mnb_count),
            accuracy_score(y_test, y_pred_svm_count),
            accuracy_score(y_test, y_pred_rf_count),
            accuracy_score(y_test, y_pred_log_reg_word2vec),
            accuracy_score(y_test, y_pred_svm_word2vec),
            accuracy_score(y_test, y_pred_rf_word2vec),
            accuracy_score(y_test, y_pred_bert_rf),
            accuracy_score(y_test, y_pred_svc_bert),   
            accuracy_score(y_test, y_pred_log_reg_bert)
    ],
    'support':[
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test),
                len(y_test)

    ]
}

results_df = pd.DataFrame(results)
results_df['Accuracy'] = results_df['Accuracy'].round(3)
print(results_df)

        Model  Accuracy  support
0   tfidf_mnb     0.597     1500
1    tfidf_lr     0.611     1500
2   tfidf_svm     0.610     1500
3    tfidf_rf     0.614     1500
4    count_lr     0.578     1500
5   count_mnb     0.602     1500
6   count_svm     0.620     1500
7    count_rf     0.609     1500
8      w2v_lr     0.586     1500
9     w2v_svm     0.583     1500
10     w2v_rf     0.563     1500
11    bert_rf     0.593     1500
12   bert_svm     0.623     1500
13    bert_lr     0.616     1500


In [51]:
# get score for every label
reports = [
    classification_report(y_test, y_pred_td_mnb, output_dict=True),
    classification_report(y_test, y_pred_td_lr, output_dict=True),
    classification_report(y_test, y_pred_td_svm, output_dict=True),
    classification_report(y_test, y_pred_td_rf, output_dict=True),
    classification_report(y_test, y_pred_log_reg_count, output_dict=True),
    classification_report(y_test, y_pred_mnb_count, output_dict=True),
    classification_report(y_test, y_pred_svm_count, output_dict=True),
    classification_report(y_test, y_pred_rf_count, output_dict=True),
    classification_report(y_test, y_pred_log_reg_word2vec, output_dict=True),
    classification_report(y_test, y_pred_svm_word2vec, output_dict=True),
    classification_report(y_test, y_pred_rf_word2vec, output_dict=True),
    classification_report(y_test, y_pred_bert_rf, output_dict=True),
    classification_report(y_test, y_pred_svc_bert, output_dict=True),
    classification_report(y_test, y_pred_log_reg_bert, output_dict=True)
]

f1_ACL = [report['ACL']['f1-score'] for report in reports]
f1_EMNLP = [report['EMNLP']['f1-score'] for report in reports]
f1_LREC = [report['LREC']['f1-score'] for report in reports]
models = [model for model in results_df['Model']]
f1_df = pd.DataFrame(
    [models, f1_ACL, f1_EMNLP, f1_LREC]
)

f1_df = f1_df.transpose()
f1_df.columns = ['Model', 'ACL', 'EMNLP', 'LREC']
f1_df[['ACL', 'EMNLP', 'LREC']] = f1_df[[ 'ACL', 'EMNLP', 'LREC']].round(2)
f1_df

,Model,ACL,EMNLP,LREC
0,tfidf_mnb,0.460568,0.525328,0.805697
1,tfidf_lr,0.48,0.523017,0.820214
2,tfidf_svm,0.48227,0.523477,0.820158
3,tfidf_rf,0.532443,0.488599,0.808923
4,count_lr,0.477341,0.479452,0.781726
5,count_mnb,0.47363,0.517647,0.809477
6,count_svm,0.532319,0.506383,0.81746
7,count_rf,0.525373,0.483563,0.802281
8,w2v_lr,0.479508,0.477178,0.783019
9,w2v_svm,0.461538,0.489022,0.777778


In [ ]:
#                Model  Accuracy
# 0                MNB  0.849167
# 1             LogReg  0.881833
# 2                SVM  0.983333
# 3     Count + LogReg  1.000000
# 4        MNB + Count  0.982833
# 5        Count + SVM  0.929000
# 6        Count + KNN  0.659333
# 7  Word2Vec + LogReg  0.614500
# 8     Word2Vec + SVM  0.607833

In [53]:
# 创建Pipeline
pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('logistic_regression', LogisticRegression(max_iter=1000, C=0.1))
])

# 拟合模型
pipeline.fit(X_train, y_train)

# 进行预测
y_pred = pipeline.predict(X_test)

# 输出结果
print("Pipeline + Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Pipeline + Logistic Regression Accuracy: 0.6086666666666667
              precision    recall  f1-score   support

         ACL       0.50      0.48      0.49       500
       EMNLP       0.50      0.53      0.52       500
        LREC       0.82      0.82      0.82       500

    accuracy                           0.61      1500
   macro avg       0.61      0.61      0.61      1500
weighted avg       0.61      0.61      0.61      1500



In [1]:
# optimiser count
count_2gram = Pipeline([('count2gram', CountVectorizer(ngram_range=(2,2))),
                        ('classifier', LogisticRegression(max_iter=1000, C=0.1))
                       ])

count_2gram.fit(X_train, y_train)
y_pred_count_2gram = count_2gram.predict(X_test)
print("count_lr_2gram:", accuracy_score(y_test, y_pred_count_2gram))

NameError: name 'Pipeline' is not defined